# Classificação Automatizada de Tomates

Este projeto tem como objetivo criar um sistema de classificação automática de tomates verdes e maduros utilizando aprendizado de máquina. O sistema é capaz de distinguir entre tomates verdes e maduros com base em imagens dos mesmos.

## Descrição do Projeto

O projeto é dividido em várias etapas:

1. **Coleta de Dados:** Foram coletadas imagens de tomates verdes e maduros da internet usando a biblioteca `bing_image_downloader`. As imagens foram organizadas em duas pastas: `tomate maduro` e `tomate verde`.

2. **Pré-processamento:** As imagens foram redimensionadas para um tamanho padrão de 200x200 pixels e normalizadas para facilitar o treinamento do modelo. Isso foi feito para garantir que todas as imagens tenham as mesmas dimensões e características.

3. **Divisão do Conjunto de Dados:** O conjunto de dados foi dividido em conjuntos de treinamento, validação e teste. As imagens foram carregadas e os rótulos foram atribuídos com base nas pastas em que estavam localizadas.

4. **Escolha do Modelo:** Um modelo de aprendizado de máquina foi escolhido e implementado usando a biblioteca `tensorflow.keras`. O modelo consiste em várias camadas convolucionais e de pooling seguidas por camadas totalmente conectadas.

5. **Treinamento do Modelo:** O modelo foi treinado usando o conjunto de treinamento e validado usando o conjunto de validação. As métricas de desempenho, como precisão e recall, foram monitoradas durante o treinamento.

6. **Avaliação do Modelo:** O modelo treinado foi avaliado usando o conjunto de teste para avaliar sua precisão e desempenho em dados não vistos.

## Requisitos de Instalação

- Python 3.x
- Bibliotecas Python: tensorflow, numpy, PIL, bing_image_downloader

In [4]:
#instalação das dependencias do projeto
!pip intall tensorflow numpy PIL bing_image_downloader

ERROR: unknown command "intall" - maybe you meant "install"


In [5]:
#Baixar as imagens de tomates verdes e maduros da internet
from bing_image_downloader import downloader

search = 'tomate maduro'
search2 = 'tomate verde'

downloader.download(search, limit=50,  output_dir='dataset', adult_filter_off=True, force_replace=False, verbose=True)
downloader.download(search2, limit=50,  output_dir='dataset', adult_filter_off=True, force_replace=False, verbose=True)

[%] Downloading Images to /home/antoniosvj/Projetos_Python/automatizacao_classificacao_tomate/dataset/tomate maduro


[!!]Indexing page: 1

[%] Indexed 35 Images on Page 1.


[%] Downloading Image #1 from https://s3.dia.es/medias/h35/h3e/8834226061342.jpg


[%] File Downloaded !

[%] Downloading Image #2 from https://3.bp.blogspot.com/-zrEYCmthErI/TpNP29en6hI/AAAAAAAAHsQ/UWKUt4e-9n4/s1600/9.jpg
[%] File Downloaded !

[%] Downloading Image #3 from https://tribunadejundiai.com.br/wp-content/uploads/2022/08/Tomates.jpg
[%] File Downloaded !

[%] Downloading Image #4 from https://pimpamfruit.com/210-thickbox_default/tomate-maduro.jpg
[%] File Downloaded !

[%] Downloading Image #5 from https://biozaki.org/wp-content/uploads/tomate-maduro-biozaki.jpg
[%] File Downloaded !

[%] Downloading Image #6 from https://fruitagash.es/372-large_default/tomate-maduro-super-oferta.jpg
[%] File Downloaded !

[%] Downloading Image #7 from https://i.ytimg.com/vi/EFWP46evlvk/maxresdefault.jpg
[%] File Downloaded !

[%] Downloading Image #8 from https://huverfruit.es/wp-content/uploads/2015/04/tomate-maduro-fruteria-online-madrid.jpg
[%] File Downloaded !

[%] Downloading Image #9 from https://donadores.pt/354-large_default/tomate-maduro.jpg
[%] File Downloaded

In [6]:
#Pré processamento das imagens
import os
from PIL import Image

def redimensionar_imagem(caminho_imagem, novo_tamanho):
    imagem = Image.open(caminho_imagem)
    imagem_redimensionada = imagem.resize(novo_tamanho)
    return imagem_redimensionada

def redimensionar_imagens(pasta_origem, novo_tamanho):
    # Percorre todas as imagens na pasta de origem
    for nome_imagem in os.listdir(pasta_origem):
        caminho_imagem_origem = os.path.join(pasta_origem, nome_imagem)
        
        try:
            # Tenta abrir a imagem
            imagem_redimensionada = redimensionar_imagem(caminho_imagem_origem, novo_tamanho)
        except Exception as e:
            print(f"Erro ao processar a imagem {caminho_imagem_origem}: {e}")
            continue
        
        # Converte a imagem para o modo RGB (se necessário)
        if imagem_redimensionada.mode == 'RGBA':
            imagem_redimensionada = imagem_redimensionada.convert('RGB')
        
        # Salva a imagem redimensionada na pasta de destino (sobrescreve a imagem original)
        caminho_imagem_destino = caminho_imagem_origem  # Salva na mesma pasta de origem
        imagem_redimensionada.save(caminho_imagem_destino)

# Definindo o novo tamanho das imagens (200x200 pixels)
novo_tamanho = (200, 200)

caminho_pasta_maduro = './dataset/tomate maduro'
redimensionar_imagens(caminho_pasta_maduro, novo_tamanho)

caminho_pasta_verde = './dataset/tomate verde'
redimensionar_imagens(caminho_pasta_verde, novo_tamanho)


In [7]:
#Normatização
import os
import numpy as np
from PIL import Image

def carregar_imagens(caminho_pasta):
    imagens = []
    for nome_imagem in os.listdir(caminho_pasta):
        caminho_imagem = os.path.join(caminho_pasta, nome_imagem)
        imagem = Image.open(caminho_imagem)
        array_imagem = np.array(imagem)
        imagens.append(array_imagem)
    return np.array(imagens)

def calcular_estatisticas(imagens):
    #calcular a média e o desvio padrão
    media = np.mean(imagens, axis=(0,1,2))
    desvio_padrao = np.std(imagens, axis=(0,1,2))
    return media, desvio_padrao

def normalizar_imagens(imagens, media, desvio_padrao):
    #normalizar imagens
    imagens_normalizadas = (imagens - media) / desvio_padrao
    return imagens_normalizadas

In [8]:
caminho_pasta_maduro = './dataset/tomate maduro'
imagens_maduro = carregar_imagens(caminho_pasta_maduro)
media_maduro, desvio_padrao_maduro = calcular_estatisticas(imagens_maduro)
imagens_normalizadas_maduro = normalizar_imagens(imagens_maduro, media_maduro, desvio_padrao_maduro)

caminho_pasta_verde = './dataset/tomate verde'
imagens_verde = carregar_imagens(caminho_pasta_verde)
media_verde, desvio_padrao_verde = calcular_estatisticas(imagens_verde)
imagens_normalizadas_verde = normalizar_imagens(imagens_verde, media_verde, desvio_padrao_verde)

In [9]:
#Treinamento do modelo

#criando array das imagens e rotulos para treinamento
import os
import numpy as np
from PIL import Image

def carregar_imagens_e_rotulos(pasta_tomate_maduro, pasta_tomate_verde):
    imagens = []
    rotulos = []

    #carregar imagens de tomates maduros
    for nome_imagem in os.listdir(pasta_tomate_maduro):
        caminho_imagem = os.path.join(pasta_tomate_maduro, nome_imagem)
        imagem = Image.open(caminho_imagem)
        imagens.append(np.array(imagem))#adicionar imagem como uma array numpy
        rotulos.append(1) #tomate maduro rotulado como 1

    #carregar imagens de tomates verdes
    for nome_imagem in os.listdir(pasta_tomate_verde):
        caminho_imagem = os.path.join(pasta_tomate_verde, nome_imagem)
        imagem = Image.open(caminho_imagem)
        imagens.append(np.array(imagem))
        rotulos.append(0) #tomate verde rotulado como 0

    return np.array(imagens), np.array(rotulos)

pasta_tomate_maduro = './dataset/tomate maduro'
pasta_tomate_verde = './dataset/tomate verde'

imagens, rotulos = carregar_imagens_e_rotulos(pasta_tomate_maduro, pasta_tomate_verde)

print("Número de imagens", len(imagens))
print("Número de rótulos: ",len(rotulos))

Número de imagens 127
Número de rótulos:  127


In [10]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Definição do modelo
modelo = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Saída binária (0 para tomate verde, 1 para tomate maduro)
])

# Compilação do modelo
modelo.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])

# Visualização da arquitetura do modelo
modelo.summary()


: 

In [ ]:
from sklearn.model_selection import train_test_split

# Divisão dos dados em treinamento, validação e teste
X_train, X_temp, y_train, y_temp = train_test_split(imagens, rotulos, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Treinamento do modelo
historico = modelo.fit(
    X_train, y_train, 
    epochs=10, batch_size=32, 
    validation_data=(X_val, y_val)
)

In [ ]:
#Testando o modelo
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Avaliação do modelo no conjunto de teste
y_pred = modelo.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# Métricas de avaliação
accuracy = accuracy_score(y_test, y_pred_binary)
report = classification_report(y_test, y_pred_binary)
conf_matrix = confusion_matrix(y_test, y_pred_binary)

print("Accuracy:", accuracy)
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


#Contribuições
Contribuições são bem-vindas! Se você quiser contribuir para este projeto, sinta-se à vontade para abrir uma issue ou enviar uma solicitação pull request.